In [24]:
#Loading the required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.autograd import Variable

import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import confusion_matrix

In [25]:
testing_set = torchvision.datasets.FashionMNIST("./data", download=True, train=False, transform=
                                               transforms.Compose([transforms.ToTensor()]))

In [26]:
test_loader = torch.utils.data.DataLoader(testing_set, batch_size=100)

In [27]:
### Creating the model and it isq named as FashionMNIST
### It contains the layers and forward function implementation
class FashionMNIST(nn.Module):

  def __init__(self):
    super(FashionMNIST, self).__init__()

    self.layer1 = nn.Sequential(
        nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
    )

    self.layer2 = nn.Sequential(
        nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2)
    )

    self.fc1 = nn.Linear(in_features=64*6*6, out_features=600)
    self.fc2 = nn.Linear(in_features=600, out_features=120)
    self.fc3 = nn.Linear(in_features=120, out_features=10)

  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = out.view(out.size(0), -1)
    out = self.fc1(out)
    out = self.fc2(out)
    out = self.fc3(out)
        
    return out

In [28]:
import glob
from google.colab import drive
drive.mount('/google_drive', force_remount = True)


#### calling the saved model using variable Z
Z = FashionMNIST()
Z.load_state_dict(torch.load("/google_drive/MyDrive/cnnmodel"))

Mounted at /google_drive


<All keys matched successfully>

In [29]:
##### Testing on the test_loader data
##### Accuraccy is calculated
batch_size = 100
label_list = []
accuracy_list = []
predictions_list = []
correct = 0

for batch_idx, (x, labels) in enumerate(test_loader):

    label_list.append(labels)
    x = x.view(batch_size, 1,28,28)
    #testing on model
    outputs = Z(x)
    predictions = torch.max(outputs, 1)[1]
    predictions_list.append(predictions)
    correct += (predictions == labels).sum()
            
    
#accuraccy          
accuracy = correct * 100 / len(testing_set)
accuracy_list.append(accuracy)

#### Question 1) part b) Classification accuracy:
print("Accuraccy is:", accuracy)

Accuraccy is: tensor(90.3700)


In [30]:
from itertools import chain 

predictions_new = [predictions_list[i].tolist() for i in range(len(predictions_list))]
labels_new = [label_list[i].tolist() for i in range(len(label_list))]
predictions_new = list(chain.from_iterable(predictions_new))
labels_new = list(chain.from_iterable(labels_new))

In [41]:
#predictions done for images and savin them to file cnn.txt
file = open("/google_drive/MyDrive/cnn.txt", 'w')
file.writelines(["%s\n" % item  for item in predictions_new])
file.close()


In [31]:
#### Question 1) part c) Confusion matrix\

import sklearn.metrics as metrics
confusion_matrix(labels_new, predictions_new)

array([[849,   0,  19,  15,   4,   1, 103,   0,   9,   0],
       [  1, 990,   1,   5,   0,   0,   2,   0,   1,   0],
       [ 15,   1, 889,   8,  35,   0,  50,   0,   2,   0],
       [ 19,  21,  16, 874,  27,   0,  41,   0,   2,   0],
       [  3,   4,  71,  21, 835,   1,  65,   0,   0,   0],
       [  0,   0,   0,   0,   0, 993,   0,   6,   0,   1],
       [107,   3,  67,  23,  62,   0, 728,   0,  10,   0],
       [  0,   0,   0,   0,   0,  22,   0, 973,   0,   5],
       [  2,   3,   4,   1,   3,   1,   4,   1, 981,   0],
       [  1,   0,   0,   0,   0,  16,   0,  58,   0, 925]])